In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dask import dataframe as dd
from dask.distributed import Client
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


/Users/nathanieldirenzo/opt/anaconda3/envs/engineering/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
client = Client()

2022-04-05 23:19:34,933 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/nathanieldirenzo/Documents/GitHub/celestial_body_size_predictor/dask-worker-space/worker-qd2_u6t_', purging
2022-04-05 23:19:34,933 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/nathanieldirenzo/Documents/GitHub/celestial_body_size_predictor/dask-worker-space/worker-f5s20tdb', purging
2022-04-05 23:19:34,934 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/nathanieldirenzo/Documents/GitHub/celestial_body_size_predictor/dask-worker-space/worker-p39c06vo', purging
2022-04-05 23:19:34,935 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/nathanieldirenzo/Documents/GitHub/celestial_body_size_predictor/dask-worker-space/worker-kw5blbvc', purging


In [ ]:
bucket_name="ndir-metis-bucket"
path=f"gs://{bucket_name}/asteroid/processed_asteroid_data.csv"

In [ ]:
df = dd.read_csv(path,
                 storage_options={"token": "secrets.json"},
                 blocksize='64MB',
                 low_memory=False)

In [ ]:
df.head()

In [ ]:
df= df.persist()

In [ ]:
df = df.compute()

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
sns.displot(df.diameter)

In [ ]:
df = df[df['diameter'] < np.quantile(df['diameter'], 0.99)]

In [ ]:
df

In [ ]:
sns.displot(df.diameter)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm');

# Feature Info
- a: semi-major axis (au)
- e: eccentricity
- i: inclination with respect to ecliptic plane
- om: longitude of the ascending node
- w: argument of perihelion
- q: perihelion distance (au)
- ad: aphelion distance (au)
- per_y: orbital period (years)
- data_arc: span of recorded data (days)
- condition_code: orbital condition code
- n_obs_used: number of observations used
- H: absolute magnitude parameter
- neo: near-earth object
- pha: physically hazardous object
- diameter: diameter (target variable)
- extent: Object bi/tri axial ellipsoid dimensions(Km)
- albedo: geometric albedo
- rot_per: rotation period (hours)
- GM: gravitational parameter. Product of mass and gravitational constant
- BV: Color index B-V magnitude difference
- UB: Color index U-B magnitude difference
- IR: Color index I-R magnitude difference
- specB: Spectral taxonomic type(SMASSII)
- specT: Spectral taxonomic type (Tholen)
- G: Magnitude slope parameter
- moid: Earth minimum orbit intersection distance
- class: asteroid orbit class
- n: mean motion (degrees/day)
- per: orbital period (days)
- ma: mean ananomly (degrees)


In [ ]:
df = df.drop(columns=['om', 'w', 'ma', 'condition_code'])

In [ ]:
df.condition_code.value_counts()